# Building GPT Model with kabyar-Corpus

#### Run by Ye Kyaw Thu
#### Lab Leader, Language Understanding Lab., Myanmar
#### Visiting Professor, NECTEC, Thailand
#### Date: 18 April 2023

## Corpus Information

In [1]:
%pwd

'/home/rnd/tool/nanoGPT'

In [2]:
%cd data/kabyar_char

/home/rnd/tool/nanoGPT/data/kabyar_char


In [4]:
!wc kabyar-corpus-ver1.0.txt

  50941  110342 3462210 kabyar-corpus-ver1.0.txt


In [6]:
!head -n 30 ./kabyar-corpus-ver1.0.txt

Title: တက်လူ့တေးသံ
By: ဇော်ဂျီ
ကြက်ဖ သာလျှင်
အာရုဏ်ရောင်လှ ၊ ဝင်းဝါကြ၏ ။
ဥဩ သာလျှင်
ရာသီနွေလ ၊ ဖူးပွင့်ကြ၏ ။
ဖားငယ် သာလျှင်
အာကာမိုးက ၊ မိုးရွာကြ၏ ။
တက်လူ သာလျှင်
မြန်မာပြည်လှ ၊ အားသစ်ရ၍
ဇေယျအောင်လံ ထူမည်တည်း ။

Title: ဤနေရာ
By: ဇော်ဂျီ
ဤနေရာတွင်
ညောင်ညိုပင်၏ ၊ မြေပြင်ခြေရင်း
မြစ်ပါးပျဉ်းသည် ၊ သက်ဆင်းလူးလွန့်
မြွေသို့တွန့်၏ ။
ဤနေရာတွင်
ပိန္နဲ့ပင်ဝယ် ၊ ရှဉ့်ရင်ပေါ့ပါး
မြီးဖားဖားသည် ၊ ရွက်ကြားခက်လက်
လျှပ်သို့လက်၏ ။
ဤနေရာတွင်
ထန်းနှစ်ပင်သည် ၊ တူယှဉ်ပြိုင်မြင့်
ရွက်ဝန်းဖွင့်၍ ၊ အကျင့်သိက္ခာ
ရှင်သို့သာတည့် ။
ဤနေရာတွင်
စေတီရှင်သည် ၊ ဖြူစင်မောက်မို့
ကြားဖူးသို့တည့် ။
ဤနေရာတွင်


## Vocab Building and Training/Validation Data Separation

I assigned kabyar-corpus filename and the updated python script is as follows:  

In [8]:
!cat ./prepare-my-char.py

"""
Prepare the Shakespeare dataset for character-level language modeling.
So instead of encoding with GPT-2 BPE tokens, we just map characters to ints.
Will save train.bin, val.bin containing the ids, and meta.pkl containing the
encoder and decoder and some other related info.
"""
import os
import pickle
import requests
import numpy as np

# download the tiny shakespeare dataset
input_file_path = os.path.join(os.path.dirname(__file__), 'kabyar-corpus-ver1.0.txt')
#if not os.path.exists(input_file_path):
#    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
#    with open(input_file_path, 'w') as f:
#        f.write(requests.get(data_url).text)

with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"voca

**Run the above python script ...**  

In [9]:
!python ./prepare-my-char.py

length of dataset in characters: 1,247,440
all the unique characters: 
 !"&'(),-./0137:=?ABCDEFGHIKLMNOPRSTUVWY_`abcdefghijklmnoprstuvwxyကခဂဃငစဆဇဈဉညဋဌဍဎဏတထဒဓနပဖဗဘမယရလဝသဟဠအဣဤဥဦဧဩဪါာိီုူေဲံ့း္်ျြွှဿ၀၁၂၃၄၅၆၇၈၉၊။၌၍၎၏႕​‌‘’“”…﻿
vocab size: 151
train has 1,122,696 tokens
val has 124,744 tokens


**Check the splitted training, validation output files:**  

In [10]:
!ls

kabyar-corpus-ver1.0.txt  meta.pkl  prepare-my-char.py	train.bin  val.bin


## Preparing the Configuration File for Training

In [12]:
!cat /home/rnd/tool/nanoGPT/config/train_kabyar_char.py

# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-kabyar-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'kabyar-char'
wandb_run_name = 'mini-gpt'

dataset = 'kabyar_char'
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# 

## Check the GPU Status

In [11]:
!nvidia-smi

Tue Apr 18 03:12:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 30%   45C    P0    58W / 300W |      0MiB / 11019MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:42:00.0 Off |                  N/A |
| 61%   

## Training GPT-2 Model with Kabyar Character Unit

In [13]:
%cd /home/rnd/tool/nanoGPT/

/home/rnd/tool/nanoGPT


In [14]:
!time python -m torch.distributed.launch --use-env train.py ./config/train_kabyar_char.py | tee train-kabyar-char.log

/home/rnd/anaconda3/envs/nanoGPT/lib/python3.8/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Overriding config with ./config/train_kabyar_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-kabyar-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'kabyar-char

## Check the model

In [17]:
!ls -lh ./out-kabyar-char/

total 124M
-rw-rw-r-- 1 rnd rnd 124M Apr 18 03:28 ckpt.pt


In [16]:
!wc ./out-kabyar-char/ckpt.pt

   423186   2410706 129382378 ./out-kabyar-char/ckpt.pt


## Testing-1 (char Model)

In [20]:
!time python -m torch.distributed.launch --use-env sample.py --out_dir=out-kabyar-char | tee test_kabyar_char.dist.log1

/home/rnd/anaconda3/envs/nanoGPT/lib/python3.8/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Overriding: out_dir = out-kabyar-char
number of parameters: 10.68M
Loading meta from data/kabyar_char/meta.pkl...

ကိုယ့်လက်ချက်တွေက လက်ရှိုက်နှိပ်တယ်
ကျားမြန်မာတွေက လက်စေးနေတယ်။

Title: ကျားမျှော်လင့်ချက်
By: နီတိ
ကော်ပျော်ရွှင်လန်းနေတဲ့
တိတ်တတ်တယ်
ကျားမျှော်လင့်ခြင်းတွေက
ပျော်ရွှင်တွေကို
ကျားသန့်နေခဲ့။
ကောင်းကင်ဟာ
သူ့နှလုံးသားတွေနဲ့
ကျားကျားခဲ့တယ်
ကျားခဲ့ရတဲ့ လက်မျှားကျားတွေထဲ
ကျားကျားထည့်ရတဲ့ စကားလုံးထဲ
သူဟာ
အားလုံးတွေနဲ့ လက်မတတ်တစ်ပုဒ်ကို
မျက်ရည်ဖွယ်ရောက်လုံး
ကျားလည်ပြုံးနေရတဲ့ အိပ်ရာဝင်/ပျံသွားတာကိ

## Testing-2 (char Model)
I changed seed value to "2023"

In [22]:
!time python -m torch.distributed.launch --use-env sample.py --out_dir=out-kabyar-char | tee test_kabyar_char.dist.log2

/home/rnd/anaconda3/envs/nanoGPT/lib/python3.8/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Overriding: out_dir = out-kabyar-char
number of parameters: 10.68M
Loading meta from data/kabyar_char/meta.pkl...

သူတို့ယောက်ျား ။
သူတို့ကိုသာ လိုညာသည်
လှောက်စရာ ကြိုးစား ။
ကိုယ်တို့ကြည့်ရှောက်ဖို့ မှောင်သား ။
သွား သူတော် ငှက်ကို သူ အုတ်အုပ်ထွေ
ပျော်ရွှင် ရောင်ခြည်မှု ။
ငှက်ကျား ပျို့ရည် ကိုယ့်မှာခ
ငှက်ငယ် ရှုတ်မပါ ။
သူတို့ကို ပြန်ပေါကြ
ကိုယ့်တို့ဆီ ချိုချိုပြန် ။
သူ့မျက်နှာ ပျင်းတော့
နှစ်ပြန်ပေါ် သက်ရောင်ထွက် ။
သူ့မျက်ရည်
သွေးစက်တော့ တောင့်တောက် ။
သူ့ကိုယ့် သူတို့နှင့်
ဘယ်သူ ဘာမျက်နှာ ဖြစ်ပေါက်
ကြည်ဖြ

## Data Preparation for BPE Unit

In [23]:
%cd data/kabyar_bpe/

/home/rnd/tool/nanoGPT/data/kabyar_bpe


In [25]:
!cat ./prepare-bpe.py

import os
import requests
import tiktoken
import numpy as np

# download the tiny shakespeare dataset
input_file_path = os.path.join(os.path.dirname(__file__), 'kabyar-corpus-ver1.0.txt')

with open(input_file_path, 'r') as f:
    data = f.read()
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode with tiktoken gpt2 bpe
enc = tiktoken.get_encoding("gpt2")
train_ids = enc.encode_ordinary(train_data)
val_ids = enc.encode_ordinary(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(os.path.dirname(__file__), 'train.bin'))
val_ids.tofile(os.path.join(os.path.dirname(__file__), 'val.bin'))



**Run above python script**

In [26]:
!python ./prepare-bpe.py

train has 3,040,974 tokens
val has 345,016 tokens


## Config File Preparation for BPE Unit

In [28]:
%cd /home/rnd/tool/nanoGPT/config

/home/rnd/tool/nanoGPT/config


In [29]:
!cat ./train_kabyar_bpe.py

# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-kabyar-bpe'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'kabyar-bpe'
wandb_run_name = 'mini-gpt'

dataset = 'kabyar_bpe'
batch_size = 32
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super necessary potentially

# on macbook also add
# dev

## Building GPT-2 Model with BPE Unit

In [31]:
%cd ..

/home/rnd/tool/nanoGPT


In [32]:
!time python -m torch.distributed.launch --use-env train.py ./config/train_kabyar_bpe.py | tee train-kabyar-bpe.log1

/home/rnd/anaconda3/envs/nanoGPT/lib/python3.8/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Overriding config with ./config/train_kabyar_bpe.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-kabyar-bpe'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'kabyar-bpe'
w

## Testing-1 with Kabyar Corpus 1.0 BPE Unit

In [35]:
!time python -m torch.distributed.launch --use-env sample.py --out_dir=out-kabyar-bpe | tee test_kabyar_bpe.dist.log1

/home/rnd/anaconda3/envs/nanoGPT/lib/python3.8/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Overriding: out_dir = out-kabyar-bpe
number of parameters: 29.94M
No meta.pkl found, assuming GPT-2 encodings...

မင်းတို့ရဲ့ ဆောက်ကိုက်နေတဲ့
ဟောင်းလေးတွေလို
တစ်နေ့တစ်ခါ နှစ်ခုခုလုံးက
ကိုယ့်အတွက်ပျက် ရွှေလက်နေတဲ့ များတယ်လို့
အချစ်တွေ များတယ်လို့ များတယ်
များတယ်
မျှားတယ်
များတယ် မျဉ်းတယ်
မျှော
---------------

ကျေးဇူးစကား တို့တန်ခူးလေး ။
ကျေးဇူးငွေ့ မုန်တိုင်းလေး ၊ တေးချိုချိုနဲ့ ။
ဘီယိုက်တို့ မွေးရပ်ရေး ၊ အမေးမြအေးကိုလည်း ။
မေးကွေးမေးကြောင်း ၊ မောင်းမဲ့လေးကလေး ။
မဗေဒါ မယ့်အတိတ်ကလေး ၊ လေပြည်သ�
----------

## Testing-2 with Kabyar Corpus 1.0 with BPE Unit

In [37]:
!time python -m torch.distributed.launch --use-env sample.py --out_dir=out-kabyar-bpe | tee test_kabyar_bpe.dist.log2

/home/rnd/anaconda3/envs/nanoGPT/lib/python3.8/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Overriding: out_dir = out-kabyar-bpe
number of parameters: 29.94M
No meta.pkl found, assuming GPT-2 encodings...

နွေသည်နေ့၏ ဆိုင်းကစားသော်ကြောင့်
သင် အဖြေများကြောင့်
ကြောက်ကြောင့်ပင် နှစ်လေးတစ်ပင်တစ်ပွင့်
တစ်ပင် ရေခြင်းသားတစ်ပွင့်
စေတနာမှန်မှု ကောက်ပင်လယ်ပြင်ပြင်
ကျောင်းသားမငြီး သူယှဉ်လမ်
---------------

သူတို့တစ်ရပ်ရဲ့ ကြမ်းမားပါ
အားလုံးကို အသစ်တစ်ခုလုံးကို လုံခြုံဖြတ်
ရိပ်လုံးလေးကို ကူးခတ်လိုက် ပြန်မလိုက်ဘူး
လူတော်ကောင်းလေးရေ ခန္ဓာကိုယ်တွေ မရှက်ဘူး
မီးပွက်လဲပြီး ရေးချင်းချင်းက�
---------------

ပေါင

Note: BPE unit နဲ့ sub-word level segmentation လုပ်ပြီး train လုပ်တဲ့အခါမှာတော့ Unknown character လိုမျိုး ပေါ်လာတာကိုတော့ တွေ့ရတယ်။  

## Reference

[1] https://github.com/karpathy/nanoGPT